In [1]:
import csv
import re

import datasets
import numpy as np
import pandas as pd
import torch
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from transformers import BertModel, BertTokenizer


In [2]:
load_embeddings = True
load_models = True


In [3]:
dataset = datasets.load_dataset("stanfordnlp/snli")
dataset.keys()


dict_keys(['test', 'validation', 'train'])

In [4]:
train = dataset["train"]
validation = dataset["validation"]
test = dataset["test"]

train = train.filter(lambda x: x["label"] != -1)
validation = validation.filter(lambda x: x["label"] != -1)
test = test.filter(lambda x: x["label"] != -1)

In [5]:
test_entailment_ids = np.where(np.array(test["label"]) == 0)[0]
test_neutral_ids = np.where(np.array(test["label"]) == 1)[0]
test_contradiction_ids = np.where(np.array(test["label"]) == 2)[0]


In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(device)


mps


In [7]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
batch_size = 256


model.to(device)
len(train), len(validation), len(test)


(549367, 9842, 9824)

In [ ]:
def extract_emb(dataset, _batch_size=batch_size):
    texts = [data["premise"] + " " + data["hypothesis"] for data in dataset]
    total_samples = len(texts)

    result = torch.zeros(total_samples, 768)

    for i in range(0, total_samples, _batch_size):
        batch_texts = texts[i : i + _batch_size]
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512,
        )
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu()

        result[i : i + len(batch_texts), :] = cls_embeddings

    return result


In [9]:
# Load embeddings
if load_embeddings:
    # test_entailment_emb = torch.from_numpy(np.load("test_entailment_emb.npy"))
    # test_neutral_emb = torch.from_numpy(np.load("test_neutral_emb.npy"))
    # test_contradiction_emb = torch.from_numpy(np.load("test_contradiction_emb.npy"))
    validation_emb = torch.from_numpy(np.load("validation_emb.npy"))
    train_emb = torch.from_numpy(np.load("train_emb.npy"))
    test_emb = torch.from_numpy(np.load("test_emb.npy"))
else:
    # test_entailment_emb = extract_emb(test_entailment)
    # test_neutral_emb = extract_emb(test_neutral)
    # test_contradiction_emb = extract_emb(test_contradiction)

    validation_emb = extract_emb(validation)
    train_emb = extract_emb(train)
    test_emb = extract_emb(test)

    # Save embeddings
    # np.save("test_entailment_emb.npy", test_entailment_emb.numpy())
    # np.save("test_neutral_emb.npy", test_neutral_emb.numpy())
    # np.save("test_contradiction_emb.npy", test_contradiction_emb.numpy())
    np.save("validation_emb.npy", validation_emb.numpy())
    np.save("train_emb.npy", train_emb.numpy())
    np.save("test_emb.npy", test_emb.numpy())


In [10]:
pca = PCA(n_components=64)

# test_entailment_emb_pca = pca.fit_transform(test_entailment_emb)
# # print(sum(pca.explained_variance_ratio_))
# test_neutral_emb_pca = pca.fit_transform(test_neutral_emb)
# # print(sum(pca.explained_variance_ratio_))
# test_contradiction_emb_pca = pca.fit_transform(test_contradiction_emb)
# # print(sum(pca.explained_variance_ratio_))

validation_emb_pca = pca.fit_transform(validation_emb)
# print(sum(pca.explained_variance_ratio_))
train_emb_pca = pca.fit_transform(train_emb)
test_emb_pca = pca.fit_transform(test_emb)


test_entailment_emb_pca = test_emb_pca[test_entailment_ids]
test_neutral_emb_pca = test_emb_pca[test_neutral_ids]
test_contradiction_emb_pca = test_emb_pca[test_contradiction_ids]

In [11]:
test_entailment_emb_pca.shape

(3368, 64)

In [12]:
n_models = 5

models = {}


class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.gru = torch.nn.GRU(64, 64, 10, bidirectional=True)
        self.dropout = torch.nn.Dropout(0.1)
        self.fc1 = torch.nn.Linear(128, 64)
        self.activation = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(64, 3)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x


if load_models:
    for i in range(n_models):
        models[i] = torch.load(f"model_{i}.pt", map_location=device)
else:
    for i in range(n_models):
        model_ = Classifier()
        model_.to(device)
        optimizer = torch.optim.AdamW(model_.parameters(), lr=1e-5)
        criterion = torch.nn.CrossEntropyLoss()

        best_accuracy = 0
        best_loss = 10000

        early_stop_cnt = 0
        prev_acc = 0

        for epoch in range(100):
            train_loss = 0
            eval_loss = 0
            model_.train()
            for j in range(0, len(train_emb_pca), batch_size):
                batch = train_emb_pca[j : j + batch_size]
                batch = torch.tensor(batch, dtype=torch.float32).to(device)
                labels = torch.tensor(train[j : j + batch_size]["label"]).to(device)

                optimizer.zero_grad()
                outputs = model_(batch)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            print(
                f"Model: {i + 1} *** Epoch {epoch} *** Loss: {train_loss / (len(train_emb_pca) / batch_size)}"
            )

            correct = 0
            total = 0
            model.eval()

            with torch.no_grad():
                for j in range(0, len(validation_emb_pca), batch_size):
                    batch = validation_emb_pca[j : j + batch_size]
                    batch = torch.tensor(batch, dtype=torch.float32).to(device)
                    labels = torch.tensor(validation[j : j + batch_size]["label"]).to(
                        device
                    )

                    output = model_(batch)
                    loss = criterion(output, labels)
                    eval_loss += loss.item()
                    _, predicted = torch.max(output.data, 1)
                    total += labels.size(0)
                    # print(predicted)
                    # print(output.shape)
                    # print(np.argmax(predicted.cpu().detach().numpy(), axis=1).shape)
                    correct += (
                        (
                            predicted.cpu().detach().numpy()
                            == labels.cpu().detach().numpy()
                        )
                        .sum()
                        .item()
                    )

            # if eval_loss / (len(validation_emb_pca) / batch_size) < best_loss:
            #     best_loss = eval_loss / (len(validation_emb_pca) / batch_size)
            #     models[i] = model_.state_dict()
            acc = 100 * correct / total
            if acc <= prev_acc:
                early_stop_cnt += 1
            else:
                prev_acc = acc
                early_stop_cnt = 0

            if acc > best_accuracy:
                best_accuracy = acc
                models[i] = model_.state_dict()

            print(
                f"Model: {i + 1} *** Epoch {epoch} *** Eval Loss: {eval_loss / (len(validation_emb_pca) / batch_size)}"
            )
            print(f"Accuracy: {100 * correct / total:.4f}")
            if early_stop_cnt >= 5:
                break
        print(f"Best acc: {best_accuracy}")

    for i in range(n_models):
        torch.save(models[i], f"model_{i}.pt")


/var/folders/5k/mff7k5qs45x_wns7wf7_b9q00000gn/T/ipykernel_17454/26246381.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  models[i] = torch.load(f"model_{i}.pt", map_lo

In [13]:
confidence = np.zeros((len(test), n_models))


for i in range(n_models):
    model_ = Classifier()
    model_.load_state_dict(models[i])
    model_.to(device)
    model_.eval()
    with torch.no_grad():
        for j in range(0, len(test), batch_size):
            batch = test_emb_pca[j : j + batch_size]
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            labels = torch.tensor(test[j : j + batch_size]["label"])

            output = torch.nn.functional.softmax(model_(batch), dim=1)

            num_out = output.shape[0]

            row_indices = torch.arange(num_out)

            confidence[j : j + num_out, i] = (
                output[row_indices, labels[:num_out]].cpu().detach().numpy()
            )


In [14]:
# k_clusters = int(
#     sum((len(test_entailment_ids), len(test_neutral_ids), len(test_contradiction_ids)))
#     * 0.02
#     // (2 * 3 - 1)
# )
k_clusters = 40

kmeans = KMeans(n_clusters=k_clusters, random_state=42, n_init="auto")

test_entailment_labels = kmeans.fit_predict(test_entailment_emb_pca)
test_neutral_labels = kmeans.fit_predict(test_neutral_emb_pca)
test_contradiction_labels = kmeans.fit_predict(test_contradiction_emb_pca)


In [15]:
test_entailment_cnt = np.zeros(k_clusters)
test_neutral_cnt = np.zeros(k_clusters)
test_contradiction_cnt = np.zeros(k_clusters)

for i in range(k_clusters):
    test_entailment_cnt[i] = np.sum(test_entailment_labels == i)
    test_neutral_cnt[i] = np.sum(test_neutral_labels == i)
    test_contradiction_cnt[i] = np.sum(test_contradiction_labels == i)


In [52]:
rankings = {
    "entailment": {},
    "neutral": {},
    "contradiction": {},
}

for i in range(k_clusters):
    rankings["entailment"][i] = {}
    rankings["neutral"][i] = {}
    rankings["contradiction"][i] = {}

    for j in range(n_models):
        temp_confidence = confidence[
            test_entailment_ids[test_entailment_labels == i], j
        ]
        temp_rank_id = np.argsort(-temp_confidence)
        temp_rank = temp_rank_id.argsort()
        rankings["entailment"][i][j] = temp_rank

        temp_confidence = confidence[test_neutral_ids[test_neutral_labels == i], j]
        temp_rank_id = np.argsort(-temp_confidence)
        temp_rank = temp_rank_id.argsort()
        rankings["neutral"][i][j] = temp_rank

        temp_confidence = confidence[
            test_contradiction_ids[test_contradiction_labels == i], j
        ]
        temp_rank_id = np.argsort(-temp_confidence)
        temp_rank = temp_rank_id.argsort()
        rankings["contradiction"][i][j] = temp_rank


In [53]:
scores = {
    "entailment": {},
    "neutral": {},
    "contradiction": {},
}

for i in range(k_clusters):
    scores["entailment"][i] = test_entailment_cnt[i] * n_models
    scores["neutral"][i] = test_neutral_cnt[i] * n_models
    scores["contradiction"][i] = test_contradiction_cnt[i] * n_models
    for j in range(n_models):
        scores["entailment"][i] -= rankings["entailment"][i][j]
        scores["neutral"][i] -= rankings["neutral"][i][j]
        scores["contradiction"][i] -= rankings["contradiction"][i][j]


In [54]:
(scores["entailment"][0])

array([506., 356., 402., 209., 237., 173., 194., 352., 328.,  32., 446.,
       332., 297., 367., 295., 336., 222., 483., 277.,  31., 139., 138.,
       520., 181.,  95., 164., 348.,  66., 435., 193., 245., 429., 172.,
       380., 204., 228., 101., 454., 266., 101., 472., 313., 219., 238.,
       378., 444., 424., 115., 161., 397., 189., 517.,  34.,  61., 421.,
       166., 303., 191.,  92.,  36., 325., 201., 474., 447.,  92., 236.,
        95., 285., 254., 259., 104., 262., 370.,  48., 268.,  42., 497.,
       414., 500., 184., 367., 295., 130., 515., 394., 337.,  17., 196.,
        41., 403., 149.,  16., 239., 470., 280., 326., 418.,  92., 201.,
       506., 149., 384., 360.,  82., 216., 510.])

In [55]:
(
    rankings["entailment"][0][0]
    + rankings["entailment"][0][1]
    + rankings["entailment"][0][2]
    + rankings["entailment"][0][3]
    + rankings["entailment"][0][4]
)

array([ 24, 174, 128, 321, 293, 357, 336, 178, 202, 498,  84, 198, 233,
       163, 235, 194, 308,  47, 253, 499, 391, 392,  10, 349, 435, 366,
       182, 464,  95, 337, 285, 101, 358, 150, 326, 302, 429,  76, 264,
       429,  58, 217, 311, 292, 152,  86, 106, 415, 369, 133, 341,  13,
       496, 469, 109, 364, 227, 339, 438, 494, 205, 329,  56,  83, 438,
       294, 435, 245, 276, 271, 426, 268, 160, 482, 262, 488,  33, 116,
        30, 346, 163, 235, 400,  15, 136, 193, 513, 334, 489, 127, 381,
       514, 291,  60, 250, 204, 112, 438, 329,  24, 381, 146, 170, 448,
       314,  20])

In [56]:
with open("scores.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["entailment", "neutral", "contradiction"])
    for i in range(k_clusters):
        writer.writerow(
            [scores["entailment"][i], scores["neutral"][i], scores["contradiction"][i]]
        )

In [57]:
with open("cluster_indices.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["entailment", "neutral", "contradiction"])
    for i in range(k_clusters):
        writer.writerow(
            [
                test_entailment_ids[test_entailment_labels == i],
                test_neutral_ids[test_neutral_labels == i],
                test_contradiction_ids[test_contradiction_labels == i],
            ]
        )


In [58]:
#  Save confidence in csv

with open("confidence.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow([f"model_{i}" for i in range(n_models)])
    for i in range(len(test)):
        writer.writerow(confidence[i])

In [59]:
# Save rankings in csv

with open("rankings.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["entailment", "neutral", "contradiction"])
    for i in range(k_clusters):
        r_e = 0
        r_n = 0
        r_c = 0

        for j in range(n_models):
            r_e += rankings["entailment"][i][j]
            r_n += rankings["neutral"][i][j]
            r_c += rankings["contradiction"][i][j]

        writer.writerow(
            [
                r_e,
                r_n,
                r_c,
            ]
        )


In [2]:
# Read cluster indices and scores using pandas

scores_df = pd.read_csv("scores.csv")
cluster_indices_df = pd.read_csv("cluster_indices.csv")
confidence_df = pd.read_csv("confidence.csv")
rankings_df = pd.read_csv("rankings.csv")


In [ ]:
low_score_ids = []
low_k = 4
k_clusters = 40


def convert_to_list(x):
    #  remove all non-numeric characters, split by space and convert to int
    x = re.sub(r"[^0-9 ]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    x = x.split(" ")
    return np.array(x, dtype=int)


# indices = np.argpartition(-cos_sim, top_k)[:top_k]
for i in range(k_clusters):
    for j in ["entailment", "neutral", "contradiction"]:
        scores_i = convert_to_list(scores_df[j][i])
        ids_i = convert_to_list(cluster_indices_df[j][i])
        rankings_i = convert_to_list(rankings_df[j][i])
        # print(len(scores_i), len(ids_i), len(rankings_i))
        assert len(scores_i) == len(ids_i)
        lowest_k = np.argpartition(-rankings_i, low_k)[:low_k]
        low_score_ids.extend(ids_i[lowest_k])

In [4]:
intersection = pd.read_csv("intersection_output_ignore_prediction.csv")

In [5]:
intersection_ids = intersection["index"].to_list()

In [6]:
#  intersection of low_score_ids and intersection_ids
low_score_intersection = list(set(low_score_ids).intersection(intersection_ids))

In [8]:
sorted(low_score_intersection)

[397,
 1162,
 1695,
 2043,
 2487,
 2511,
 2971,
 4000,
 4034,
 4773,
 5189,
 5539,
 5991,
 7215,
 7945,
 9490]

In [10]:
len(low_score_intersection)

16

In [9]:
len(low_score_intersection) / len(intersection_ids) * 100


3.404255319148936